# Assignment - Deconstruction

**Targets and patterns.**

For an overview of assignment, see `assign1.ipynb`.

## Review of tuple/list syntax in assignment targets

### Tuple syntax

Recall that `x, y` is syntactically a tuple as it appears in these statements, but no tuple is created, *not even in principle*, because it is given as the target of assignment:

In [1]:
x, y = [10, 20]

In [2]:
print(f'{x=}, {y=}')

x=10, y=20


In [3]:
for x, y in [[30, 40], [50, 60]]:
    print(f'{x=}, {y=}')

x=30, y=40
x=50, y=60


`(x, y)` may be written in place of `x, y`, in both the assignment statement and the `for` loop shown above, and this has the same meaning. After all, in this context, both `x, y` and `(x, y)` are syntactically tuples.

### List syntax (equivalent)

Recall also that it is sometimes more convenient to use list syntax, so `[x, y]` is syntactically a list as it appears in these statements, but no list is created (except that the right-hand side coincidentally happens to be a list), *not even in principle*, because it is given as the target of assignment:

In [4]:
[x, y] = [11, 22]

In [5]:
print(f'{x=}, {y=}')

x=11, y=22


In [6]:
for x, y in [[33, 44], [55, 66]]:
    print(f'{x=}, {y=}')

x=33, y=44
x=55, y=66


### Nested tuple/list syntax

Finally, recall that nested tuple/list expressions are likewise permitted, and these are syntactically nested tuples/lists, but semantically they do not entail the creation of any list or tuple, *not even in principle*. Instead, what is happening--in all these cases--is that, informally speaking, we are specifying the *shape* of the data being deconstructed:

In [7]:
w, [x, (y, z)], t = [10, [20, [30, 40]], 50]

In [8]:
print(f'{w=}, {x=}, {y=}, {z=}, {t=}')

w=10, x=20, y=30, z=40, t=50


In [9]:
for w, [x, (y, z)], t in [1, [2, [3, 4]], 5], [6, [7, [8, 9]], 10]:
    print(f'{w=}, {x=}, {y=}, {z=}, {t=}')

w=1, x=2, y=3, z=4, t=5
w=6, x=7, y=8, z=9, t=10


`w, [x, (y, z)], t` is syntactically a tuple of:

- the name `w`
- a list of:
  - the name `x`
  - a tuple of:
    - the name `y`
    - the name `z`
- the name `t`

But no list or tuple is created. This is instead a way for the left-hand side of an assignment to specify, statically, the structure that the object appearing on the right-hand side will be expected, dynamically (i.e., at runtime), to have. This is so that object can be picked apart appropriately to assign the desired pieces to the variables.

### The left-side structure is static, the right-side structure is dynamic

Whenever the object the right-hand side *evaluates to at runtime* is not iterable or does not have the correct number of elements, or its elements that correspond to "tuples" or "lists" on the left-hand side are not iterable or do not have the correct number of elements, etc., we get an error.

In [10]:
w, [x, (y, z)], t = [10, [20, [30, 40, 41]], 50]

ValueError: too many values to unpack (expected 2)

### In `match`-`case`

`match`-`case` has more powerful pattern matching than other forms of assignment. This will be reviewed and further explored below. But the above examples apply to `match`-`case` as well:

In [11]:
def try_match(obj):
    match obj:
        case w, [x, (y, z)], t:
            print(f'{obj!r} matched. Got {w=}, {x=}, {y=}, {z=}, {t=}.')
        case _:
            print(f'{obj!r} did not match.')  # Well, it matched this catch-all pattern.

In [12]:
try_match([10, [20, [30, 40]], 50])

[10, [20, [30, 40]], 50] matched. Got w=10, x=20, y=30, z=40, t=50.


In [13]:
try_match([10, [20, [30, 40, 41]], 50])

[10, [20, [30, 40, 41]], 50] did not match.


### Why "not even in principle"

Note that all of the above are different from a situation where creating a list or tuple is specified but is likely to be optimized away, as on the *right-hand side* of:

In [14]:
p, q = 100, 200

On the *left-hand side* of that assignment statement, the reason no actual tuple `p, q` is created is not that any operation is optimized away, but that there is not even any way to *make sense* out of creating such a tuple.

(`p` and `q` need not, and in this case do not, even exist yet. Furthermore, tuples are immutable, so we cannot assign values *into* them.)

## `match`-`case` patterns

We begin with review, then explore what it means when what is *syntactically* a call expression--as if to construct an object by calling a class--appears as a pattern in `match`-`case`, and the meaning of keyword and positional arguments in such a pattern.

In [15]:
from fractions import Fraction
from numbers import Real

In [16]:
Real()  # Abstract class, can't instantiate.

TypeError: Can't instantiate abstract class Real with abstract methods __abs__, __add__, __ceil__, __eq__, __float__, __floor__, __floordiv__, __le__, __lt__, __mod__, __mul__, __neg__, __pos__, __pow__, __radd__, __rfloordiv__, __rmod__, __rmul__, __round__, __rpow__, __rtruediv__, __truediv__, __trunc__

In [17]:
def describe(n):
    match n:
        case 42:
            print(f'{n} is said to be the answer to an important question.')
        case 76:
            print(f'{n} is a pretty cool integer.')
        case int():  # Does not "construct" an int, which would always be 0.
            print(f'{n} is some integer or other.')
        case Real():  # Does not "construct" a Real, which could never succeed.
            print(f'{n} is not an integer, but a real number of some sort.')
        case _:
            print(f'{n} is not a real number.')

In [18]:
for thing in 42, 76, 100, 2.5, Fraction(5, 2), 4j, 'foo':
    describe(thing)

42 is said to be the answer to an important question.
76 is a pretty cool integer.
100 is some integer or other.
2.5 is not an integer, but a real number of some sort.
5/2 is not an integer, but a real number of some sort.
4j is not a real number.
foo is not a real number.


What happens if we try to extract 42 and 76 to named constants?

In [19]:
THE_ANSWER = 42

In [20]:
COOL_INTEGER = 76

In [21]:
def describe2(n):
    match n:
        case THE_ANSWER:
            print(f'{n} is said to be the answer to an important question.')
        case COOL_INTEGER:
            print(f'{n} is a pretty cool integer.')
        case int():  # Does not "construct" an int, which would always be 0.
            print(f'{n} is some integer or other.')
        case Real():  # Does not "construct" a Real, which could never succeed.
            print(f'{n} is not an integer, but a real number of some sort.')
        case _:
            print(f'{n} is not a real number.')

SyntaxError: name capture 'THE_ANSWER' makes remaining patterns unreachable (2454158040.py, line 3)

In [22]:
def describe2(n):
    match n:
        case THE_ANSWER:
            print(f'{n} is said to be the answer to an important question.')

In [23]:
describe2(42)

42 is said to be the answer to an important question.


In [24]:
describe2(41)

41 is said to be the answer to an important question.


In [25]:
THE_ANSWER

42

In [26]:
match 41:
    case THE_ANSWER:
        print('Hmm...')

Hmm...


In [27]:
THE_ANSWER

41

In [28]:
def f(): 
    THE_ANSWER = 4

In [29]:
f()

In [30]:
THE_ANSWER

41

In [31]:
from types import SimpleNamespace

In [32]:
CONSTANTS = SimpleNamespace(THE_ANSWER=42, COOL_INTEGER=76)

In [33]:
CONSTANTS

namespace(THE_ANSWER=42, COOL_INTEGER=76)

In [34]:
def describe3(n):
    match n:
        case CONSTANTS.THE_ANSWER:
            print(f'{n} is said to be the answer to an important question.')
        case CONSTANTS.COOL_INTEGER:
            print(f'{n} is a pretty cool integer.')
        case int():  # Does not "construct" an int, which would always be 0.
            print(f'{n} is some integer or other.')
        case Real():  # Does not "construct" a Real, which could never succeed.
            print(f'{n} is not an integer, but a real number of some sort.')
        case _:
            print(f'{n} is not a real number.')

In [35]:
for thing in 42, 76, 100, 2.5, Fraction(5, 2), 4j, 'foo':
    describe3(thing)

42 is said to be the answer to an important question.
76 is a pretty cool integer.
100 is some integer or other.
2.5 is not an integer, but a real number of some sort.
5/2 is not an integer, but a real number of some sort.
4j is not a real number.
foo is not a real number.


## Keyword arguments in `match`-`case` patterns

In [36]:
match CONSTANTS: 
    case SimpleNamespace(): 
        print('Matched')
    case _: 
        print('Did not match')

Matched


In [37]:
CONSTANTS == SimpleNamespace()

False

In [38]:
match CONSTANTS: 
    case Real(): 
        print('Matched')
    case _: 
        print('Did not match')

Did not match


In [39]:
match CONSTANTS: 
    case SimpleNamespace(THE_ANSWER=41): 
        print('Matched')
    case _: 
        print('Did not match')

Did not match


In [40]:
match CONSTANTS: 
    case SimpleNamespace(THE_ANSWER=42): 
        print('Matched')
    case _: 
        print('Did not match')

Matched


In [41]:
match CONSTANTS: 
    case SimpleNamespace(THE_ANSWER=42, COOL_INTEGER=32): 
        print('Matched')
    case _: 
        print('Did not match')

Did not match


In [42]:
match CONSTANTS: 
    case SimpleNamespace(THE_ANSWER=42, COOL_INTEGER=76): 
        print('Matched')
    case _: 
        print('Did not match')

Matched


In [43]:
match CONSTANTS: 
    case SimpleNamespace(THE_ANSWER=42, COOL_INTEGER=76, EXTRA_INTEGER=9): 
        print('Matched')
    case _: 
        print('Did not match')

Did not match


In [44]:
match CONSTANTS: 
    case SimpleNamespace(THE_ANSWER=42, COOL_INTEGER=ci): 
        print('Matched')
    case _: 
        print('Did not match')

Matched


In [45]:
ci

76

In [46]:
s = [1, 'two', CONSTANTS, SimpleNamespace(plevel=9001, second=CONSTANTS)]

In [47]:
match s: 
    case [Real(), 
          _, 
          SimpleNamespace(), 
          SimpleNamespace(second=SimpleNamespace(COOL_INTEGER=76, THE_ANSWER=answer))]: 
        print('Complex pattern match')

Complex pattern match


In [48]:
answer

42

In [49]:
match iter(s): 
    case [Real(), 
          _, 
          SimpleNamespace(), 
          SimpleNamespace(second=SimpleNamespace(COOL_INTEGER=76, THE_ANSWER=answer))]: 
        print('Complex pattern match') 

In [50]:
elem1, elem2, elem3, elem4 = iter(s)

In [51]:
elem1, elem2, elem3, elem4

(1,
 'two',
 namespace(THE_ANSWER=42, COOL_INTEGER=76),
 namespace(plevel=9001, second=namespace(THE_ANSWER=42, COOL_INTEGER=76)))

In [52]:
from algoviz.bobcats import Bobcat, FierceBobcat

In [53]:
my_bobcats = [Bobcat('Mr.Frumbles'), Bobcat('SirPounce'), FierceBobcat('OttovonMeow', 9002)]

In [54]:
match my_bobcats: 
    case [Bobcat(), 
          Bobcat(),
          Bobcat(fierceness=f)]: 
        print('Bobcats found, RUUUUN')

Bobcats found, RUUUUN


In [55]:
f

9002

In [56]:
match my_bobcats: 
    case [Bobcat(), 
          Bobcat(),
          Bobcat(FIERCENESS_CUTOFF=fc)]:
        if fc > 8999:
            print("What 9000!")

What 9000!


In [57]:
match my_bobcats: 
    case [Bobcat(), 
          Bobcat(FIERCENESS_CUTOFF=fc),
          Bobcat()]:
        if fc > 8999:
            print("What 9000!")

In [58]:
match my_bobcats: 
    case [Bobcat(), 
          Bobcat(),
          Bobcat(FIERCENESS_CUTOFF=fc)] if fc > 8999:
        print("What 9000!")

What 9000!


In [59]:
match my_bobcats: 
    case [Bobcat(), 
          Bobcat(),
          Bobcat(FIERCENESS_CUTOFF=fc) as scary_bobcat]:
        if scary_bobcat.fierceness > 9000: 
            print("It's over 9000!")

It's over 9000!


## Positional arguments in `match`-`case` patterns

In [60]:
match my_bobcats:  # With a keyword argument, first.
    case [Bobcat(name=name), 
          Bobcat(),
          Bobcat()]:
        print(name)

Mr.Frumbles


In [61]:
# This works because we defined __match_args__ in Bobcat.
match my_bobcats:
    case [Bobcat(name), 
          Bobcat(),
          Bobcat()]:
        print(name)

Mr.Frumbles


In [62]:
match my_bobcats:
    case [Bobcat(), 
          Bobcat(),
          Bobcat(name, fierceness)]:
        print(name)
        print(fierceness)

TypeError: Bobcat() accepts 1 positional sub-pattern (2 given)

In [63]:
# This works because we defined __match_args__ in FierceBobcat.
match my_bobcats:
    case [Bobcat(), 
          Bobcat(),
          FierceBobcat(name, fierceness)]:
        print(name)
        print(fierceness)

OttovonMeow
9002


In [64]:
class OldWidget: 
    
    def __init__(self, name, color): 
        self.name = name
        self.color = color
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.name!r}, {self.color!r})'
    
    @property
    def fullname(self): 
        return f'{self.color} {self.name}'

In [65]:
OldWidget('lightsaber', 'blue')

OldWidget('lightsaber', 'blue')

In [66]:
OldWidget('lightsaber', 'red').fullname

'red lightsaber'

In [67]:
anakin_lightsaber = OldWidget('lightsaber', 'blue')

In [68]:
match anakin_lightsaber: 
    case OldWidget(name): 
        print("Your father's 'light' saber")

TypeError: OldWidget() accepts 0 positional sub-patterns (1 given)

In [69]:
class Widget: 
    
    __match_args__ = ('name', 'color')
    
    def __init__(self, name, color): 
        self.name = name
        self.color = color
    
    def __repr__(self): 
        return f'{type(self).__name__}({self.name!r}, {self.color!r})'
    
    @property
    def fullname(self): 
        return f'{self.color} {self.name}'

In [70]:
Widget('lightsaber', 'blue')

Widget('lightsaber', 'blue')

In [71]:
Widget('lightsaber', 'red').fullname

'red lightsaber'

In [72]:
anakin_lightsaber = Widget('lightsaber', 'blue')

In [73]:
match anakin_lightsaber: 
    case Widget(name): 
        print("Your father's 'light' saber")

Your father's 'light' saber


In [74]:
name

'lightsaber'

In [75]:
mace_lightsaber = Widget('bad-ass-lightsaber', 'purple')

In [76]:
match mace_lightsaber: 
    case Widget(_, 'purple', fullname=fn): 
        print(f"Found Mace Windu's {fn}")

Found Mace Windu's purple bad-ass-lightsaber
